# db 가져오기

In [23]:
import requests
from bs4 import BeautifulSoup
import time
from urllib.robotparser import RobotFileParser
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
import re

headers = {'user-agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"}

def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])
    
def download(url, params={}, headers={}, method='GET', limit=3):
    method = method.upper()
    if canfetch(url) == False:
        print('[Error] ' + url)
    else: #bot 접근 못하게 막은데있어어 else뺌
        try:
            resp = requests.request(method, url,
                   params=params if method=='GET' else {},
                   data=params if method=='POST' else {},
                   headers=headers)
            resp.raise_for_status()
        except HTTPError as e:
            if limit > 0 and e.response.status_code >= 500: # 500 에러 재귀 ;너무많은 트래픽 안생기도록
                print(limit)
                time.sleep(300) # => random, 1초에 1번
                resp = download(url, params, headers, method, limit-1)
            else:
                print('[{}] '.format(e.response.status_code) + url)
                print(e.response.status_code)
                print(e.response.reason)
                print(e.response.headers)
        return resp

In [ ]:
startrow =1
endrow = 1000
api_key = '1cbfb2550a5a4dd8820f'
url = 'http://openapi.foodsafetykorea.go.kr/api/' + api_key + '/I2790/json/' + str(startrow) + '/' + str(endrow)
resp = download(url, headers = headers,method= 'GET')
result = resp.json()
total = int(result['I2790']['total_count'])
de = total%1000

import sqlite3
con = sqlite3.connect('ex.db')
cur = con.cursor()

cur.executescript('''
    DROP TABLE IF EXISTS factor;
    CREATE TABLE factor(
        NUM           INTEGER NOT NULL,
        DESC_KOR      TEXT NOT NULL ,
        FOOD_GROUP    TEXT ,
        FOOD_CD      TEXT ,
        NUTR_CONT1     FLOAT DEFAULT 0.0,
        NUTR_CONT2     FLOAT DEFAULT 0.0,
        NUTR_CONT3     FLOAT DEFAULT 0.0,
        NUTR_CONT4   FLOAT DEFAULT 0.0,
        NUTR_CONT5   FLOAT DEFAULT 0.0,
        NUTR_CONT6   FLOAT DEFAULT 0.0,
        NUTR_CONT7   FLOAT DEFAULT 0.0,
        NUTR_CONT8   FLOAT DEFAULT 0.0,
        NUTR_CONT9   FLOAT DEFAULT 0.0,
        SERVING_SIZE   INTERGER ,
        MAKER_NAME    TEXT ,
        BGN_YEAR      TEXT ,
        SAMPLING_MONTH_NAME     TEXT ,
        SUB_REF_NAME      TEXT,
        SAMPLING_REGION_NAME   TEXT ,
        GROUP_NAME       TEXT ,
        RESEARCH_YEAR INTERGER DEFAULT 0,
        SAMPLING_REGION_CD  INTERGER DEFAULT 0,
        SAMPLING_MONTH_CD  TEXT ,
        ANIMAL_PLANT    TEXT 
    );
''')

In [ ]:
for _ in result['I2790']['row']:
    cur.execute('''INSERT INTO factor(NUM,DESC_KOR ,
        FOOD_GROUP ,FOOD_CD ,NUTR_CONT1 ,
        NUTR_CONT2, NUTR_CONT3, NUTR_CONT4, NUTR_CONT5,
        NUTR_CONT6, NUTR_CONT7,  NUTR_CONT8, NUTR_CONT9,
        SERVING_SIZE  ,MAKER_NAME , BGN_YEAR, SAMPLING_MONTH_NAME  ,
        SUB_REF_NAME, SAMPLING_REGION_NAME, GROUP_NAME  ,
        RESEARCH_YEAR, SAMPLING_REGION_CD ,SAMPLING_MONTH_CD, ANIMAL_PLANT) 
        VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''',[
        _['NUM'],_['DESC_KOR'] ,
        _['FOOD_GROUP'] ,_['FOOD_CD'] ,_['NUTR_CONT1'] ,
        _['NUTR_CONT2']  ,_['NUTR_CONT3']  ,_['NUTR_CONT4']  , _['NUTR_CONT5'],
        _['NUTR_CONT6']   ,_['NUTR_CONT7'],  _['NUTR_CONT8'], _['NUTR_CONT9'],
        _['SERVING_SIZE']  ,_['MAKER_NAME'] , _['BGN_YEAR']     ,_['SAMPLING_MONTH_NAME']  ,
        _['SUB_REF_NAME'] , _['SAMPLING_REGION_NAME'] , _['GROUP_NAME']  ,
        _['RESEARCH_YEAR'] ,_['SAMPLING_REGION_CD'] ,_['SAMPLING_MONTH_CD'],_['ANIMAL_PLANT']])
con.commit()

In [ ]:
for i in range(int(total/1000)):
    print("Count : ",i)
    startrow+= 1000
    if i==int(total/1000)-1:
        endrow += de
    else:
        endrow += 1000
    url = 'http://openapi.foodsafetykorea.go.kr/api/' + api_key + '/I2790/json/' + str(startrow) + '/' + str(endrow)
    resp = download(url, headers = headers,method= 'GET')
    result = resp.json()
    
    for _ in result['I2790']['row']:
        cur.execute('''INSERT INTO factor(NUM,DESC_KOR  ,
            FOOD_GROUP ,FOOD_CD ,NUTR_CONT1 ,
            NUTR_CONT2, NUTR_CONT3, NUTR_CONT4, NUTR_CONT5,
            NUTR_CONT6, NUTR_CONT7,  NUTR_CONT8, NUTR_CONT9,
            SERVING_SIZE  ,MAKER_NAME , BGN_YEAR, SAMPLING_MONTH_NAME  ,
            SUB_REF_NAME, SAMPLING_REGION_NAME, GROUP_NAME  ,
            RESEARCH_YEAR, SAMPLING_REGION_CD ,SAMPLING_MONTH_CD, ANIMAL_PLANT) 
            VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''',[
            _['NUM'],_['DESC_KOR'] ,
            _['FOOD_GROUP'] ,_['FOOD_CD'] ,_['NUTR_CONT1'] ,
            _['NUTR_CONT2']  ,_['NUTR_CONT3']  ,_['NUTR_CONT4']  , _['NUTR_CONT5'],
            _['NUTR_CONT6']   ,_['NUTR_CONT7'],  _['NUTR_CONT8'], _['NUTR_CONT9'],
            _['SERVING_SIZE']  ,_['MAKER_NAME'] , _['BGN_YEAR']     ,_['SAMPLING_MONTH_NAME']  ,
            _['SUB_REF_NAME'] , _['SAMPLING_REGION_NAME'] , _['GROUP_NAME']  ,
            _['RESEARCH_YEAR'] ,_['SAMPLING_REGION_CD'] ,_['SAMPLING_MONTH_CD'],_['ANIMAL_PLANT']])
    time.sleep(1)
    con.commit()
    time.sleep(1)

In [ ]:
cur.close()
con.close()

# 메인 화면에서만 가져온거

In [33]:
from selenium import webdriver
driver = webdriver.Chrome('/Users/Gyu/Desktop/mywork/KData/chromedriver')
driver.get('http://xn--hg3bqa693eb6a.com/product/list.html?cate_no=24')

In [51]:
from selenium import webdriver
driver = webdriver.Chrome('/Users/Gyu/Desktop/mywork/KData/chromedriver')
driver.get('http://xn--hg3bqa693eb6a.com/product/list.html?cate_no=24')

page = 1
url = 'http://xn--hg3bqa693eb6a.com/product/list.html?cate_no=24&page='
driver.get(url+str(page))
# driver.execute_script('console.log("hi")')
leng = len(driver.find_elements_by_class_name('other'))

for r in range(leng+1):    
    for i, _ in enumerate(driver.find_elements_by_css_selector('img')):
        if _.get_attribute('src').split('/')[4]=='product':
            urllib.request.urlretrieve(_.get_attribute('src'),"./img/" + str(r) + 'bakery' +str(i)+'.jpg')
    if r == leng:
        break
    page+=1
    driver.get(url+str(page))
    time.sleep(2)

In [37]:
driver.find_element_by_css_selector('img').get_attribute('src').split('/')

['http:',
 '',
 'xn--hg3bqa693eb6a.com',
 'web',
 'upload',
 'category',
 'logo',
 '28a47eadbf2e3fc961300b61118ae4cd_3_top.jpg']

In [58]:
# download('https://www.panhonesta.com/menu',headers=headers,method='GET')

<Response [200]>

# 상품 세부 사진 가져오기

In [14]:
from selenium import webdriver
# driver = webdriver.Chrome('/Users/Gyu/Desktop/mywork/KData/chromedriver')
driver.get('http://xn--hg3bqa693eb6a.com/product/list.html?cate_no=24')
total= int(driver.find_element_by_css_selector('p[class=prd_count] > strong').text.strip())
total
# total : 메인 화면에 30개라고 나와있음

30

In [16]:
driver.close()

InvalidSessionIdException: Message: invalid session id


In [9]:
url = 'http://xn--hg3bqa693eb6a.com/product/%EC%95%BC%EB%AF%B8%EC%9A%94%EB%B0%80-8%E7%84%A1-%ED%9D%91%EC%9E%84%EC%9E%90-%ED%81%AC%EB%A6%BC-%EC%8C%80%EB%B9%B5-%EC%8C%80%EB%B9%B5%EB%B9%84%EA%B1%B4%EB%B9%B5%EB%B9%84%EA%B1%B4%EC%8C%80%EB%B9%B5%EA%B1%B4%EA%B0%95%EB%B9%B5%EB%8B%A4%EC%9D%B4%EC%96%B4%ED%8A%B8%EB%B9%B5%EB%B9%84%EA%B1%B4/22/category/24/display/1/#none'
# 첫번째 상품 상세 url
driver.get(url)
driver.find_element_by_css_selector('span[class=next] > a').click()

In [6]:
import urllib
for i, _ in enumerate(driver.find_elements_by_css_selector('img[class=ThumbImage]')):
    urllib.request.urlretrieve(_.get_attribute('src'),"./img/"+ 'ba' +str(i)+'.jpg')

In [ ]:
driver.close()

In [22]:
url = 'https://www.google.com/search?q=가지&tbm=isch&tbs=sur%3Afmc&rlz=1C1NDCM_koKR830KR830&hl=ko&sa=X&ved=0CAEQpwVqFwoTCKCn6q2_9eoCFQAAAAAdAAAAABAC&biw=1899&bih=903'
download(url,headers=headers,method='GET')

[Error] https://www.google.com/search?q=가지&tbm=isch&tbs=sur%3Afmc&rlz=1C1NDCM_koKR830KR830&hl=ko&sa=X&ved=0CAEQpwVqFwoTCKCn6q2_9eoCFQAAAAAdAAAAABAC&biw=1899&bih=903


In [24]:
download(url = 'https://pixabay.com/images/search/%EC%82%AC%EA%B3%BC/?cat=food',headers=headers, method='GET')

[Error] https://pixabay.com/images/search/%EC%82%AC%EA%B3%BC/?cat=food
